In [148]:
import requests
import json
import pandas as pd
import sys
basepath = '/'.join(os.getcwd().split("/")[:-1])
if basepath not in sys.path:
    sys.path.append(basepath)

from becp import projections
import plotly.express as px

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [165]:
area = 75999.49

design_type = 'ApartmentHighRise'

params = [
    {
    'state': 'MA',
    'climate_zone': '5A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': design_type,
            'area': area,
            'heating_fuel': 'natural gas',
            'dhw_fuel': 'natural gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': 'STD2016'
        },     
        ]
    },
    {
    'state': 'MA',
    'climate_zone': '5A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': design_type,
            'area': area,
            'heating_fuel': 'electricity',
            'dhw_fuel': 'electricity',
            'heating_cop': 3.5,
            'dhw_cop': 3.5,
            'ashrae_standard': 'STD2016'
        },     
        ]
    },
    ]

host =  'https://akf-becp-pyapi.herokuapp.com/'
host = 'http://127.0.0.1:5001'
endpoint = 'get_projection_from_reference_buildings'
payload = json.dumps(params)


results = [requests.get(f'{host}/{endpoint}/?params={json.dumps(p)}').json() for p in params]

results

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [150]:


dfs = []
for res in results:

    enduses = pd.DataFrame(res['enduses']['enduses_absolute_kbtu'])
    factors = pd.DataFrame(res['projection_factors'])


    hvac_enduses_list = [
        'Cooling', 'Fans', 'Heat Rejection', 'Heating',  'Pumps', 'Water Systems'
    ]

    hvac_enduses = enduses.loc[enduses['enduse'].isin(hvac_enduses_list)].groupby('fuel').sum()
    non_hvac_enduses = enduses.loc[~enduses['enduse'].isin(hvac_enduses_list)].groupby('fuel').sum()

    gas_factor = 0.05311 # kg per kbtu

    factors.set_index('year')['lrmer_co2e_kg_mwh'].to_dict() # kg per mwh

    factors['kg_per_kbtu'] = factors['lrmer_co2e_kg_mwh'] / 3412
    coefficients = factors.set_index('year')['kg_per_kbtu'].to_dict()


    year_results = []
    for year, coeff in coefficients.items():

        non_hvac_gas = 0
        hvac_gas = 0
        non_hvac_elec = 0
        hvac_elec = 0

        if "Natural Gas" in non_hvac_enduses.index:
            non_hvac_gas = (non_hvac_enduses.loc['Natural Gas'] * gas_factor)[0] / area
        if "Electricity" in hvac_enduses.index:
            non_hvac_elec = (non_hvac_enduses.loc['Electricity'] * coeff)[0] / area
        if "Natural Gas" in hvac_enduses.index:
            hvac_gas = (hvac_enduses.loc['Natural Gas'] * gas_factor)[0] / area
        if "Electricity" in hvac_enduses.index:
            hvac_elec = (hvac_enduses.loc['Electricity'] * coeff)[0] / area

        year_results.append({
            'year': year,
            'hvac_gas': hvac_gas,
            'non_hvac_gas': non_hvac_gas,
            'hvac_elec': hvac_elec,
            'non_hvac_elec': non_hvac_elec
        })


    projection_df = pd.DataFrame(year_results)
    projection_df['hvac_kg_per_sf'] = projection_df['hvac_gas'] + projection_df['hvac_elec']
    projection_df['non_hvac_kg_per_sf'] = projection_df['non_hvac_gas'] + projection_df['non_hvac_elec']
    projection_df['total_kg_per_sf'] = projection_df['hvac_kg_per_sf'] + projection_df['non_hvac_kg_per_sf']

    projection_df = projection_df.set_index('year')[['hvac_kg_per_sf', 'non_hvac_kg_per_sf', 'total_kg_per_sf']]

    dfs.append(projection_df)





In [151]:
hvac_results = pd.concat([x['hvac_kg_per_sf'] for x in dfs], axis=1)
non_hvac_results = pd.concat([x['non_hvac_kg_per_sf'] for x in dfs], axis=1)
total_results = pd.concat([x['total_kg_per_sf'] for x in dfs], axis=1)



In [152]:
total_results.columns = [num for num, line in enumerate(total_results.columns)]

px.line(total_results)


In [153]:
hvac_results.columns = [num for num, line in enumerate(hvac_results.columns)]

px.line(hvac_results)



In [154]:
px.area(dfs[0].iloc[:,:2], title = 'gas heating OCI')


In [155]:
## todo : show by fuel

px.area(dfs[1].iloc[:,:2], title = 'elec heating OCI')


proj = [
    projections.get_projection_from_reference_buildings({
    'state': 'NY',
    'climate_zone': '4A',
    'projection_case': case[2],
    'design_areas': [
        {
            'type': 'SmallOffice',
            'area': 200000,
            'heating_fuel': case[1],
            'dhw_fuel': case[1],
            'heating_cop': case[0],
            'dhw_cop': case[0],
            'ashrae_standard': '90.1-2016'
        },
        {
            'type': 'MediumOffice',
            'area': 10000,
            'heating_fuel': 'Natural Gas',
            'dhw_fuel': 'Natural Gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': '90.1-2007'
        },
    ]
}) for case in cases]